In [3]:
import numpy as np
import sys
import os
import textwrap
from datetime import datetime as dt
import calendar
import texts as tx

In [6]:
import sys,time

string = """ H opefully
 A ll
 R espectful
 V isitors
 E njoy
 S ome
 T ipping"""

typing_speed = 390

def slow_type(t):
    for l in t:
        sys.stdout.write(l)
        sys.stdout.flush()
        time.sleep(10.0/typing_speed)
    print('\n')

slow_type(string)

 H opefully
 A ll
 R espectful
 V isitors
 E njoy
 S ome
 T ipping



In [ ]:
STAFF = [
    'diego',
    'maike',
    'grogu',
    'sina',
    'taro',
    'grace',
    'anna bendel',
    'charlotte',
    'isabelle',
    'jessica',
    'anna',
    'juliana',
    'lea',
    'lino',
    'maja',
    'margaretha',
    'marilin',
    'matthias',
    'micaela',
    'mona',
    'anisa',
    'rieka',
    'sabina',
    'sandra',
    'santiago',
    'therese',
    'utz',
    'yunus',
    'veronica',
    'karolina',
    'imer'
]


def is_not_int(var):
    try:
        int(var)
        return False
    except:
        return True

def wich_one(name):
    hit =[]
    for s in STAFF:
        if name in s:
            hit.append(s)

    if len(hit) > 1:
        print('wich one?')
        for i, s in enumerate(hit):
            print(f'{i+1}. {s}')
            
        result = input('--> select: ')
        
        while is_not_int(result):
            result = input('--> select: ')

        return hit[int(result) - 1]

    else:
        return name


file_path = []

root_dir = './json/'
for root, dirs, files in os.walk(root_dir, onerror=None):
    for filename in files:
        if not any(s in filename for s in ['LOG', 'checkpoint', 'DS', 'edited']):
            if 'WHOLE' in filename:
                filename = os.path.join(root, filename)
                if int(filename[7:11]) == 2024:
                    if int(filename[12:14]) >= 9:
                        file_path.append(filename)
                    elif int(filename[12:14]) == 8 and int(filename[15:17]) >= 17:
                        file_path.append(filename)
                elif int(filename[7:11]) >= 2025:
                    file_path.append(filename)


staff_tip = [[] for i in range(len(STAFF))]
staff_date = [[] for i in range(len(STAFF))]


for file in file_path:
    f = open(file)
    jData = json.loads(f.read())

    try:
        for i, staff in enumerate(jData['staff']):
            staff_tip[STAFF.index(staff)].append(jData['tip'][i])
            staff_date[STAFF.index(staff)].append(jData['timestamp'])

    except KeyError:
        print('skip file', file[7:])

for i, j in enumerate(staff_tip):
    print(staff_tip[i], staff_date[i])

In [ ]:
def sorterkey(line):
    year = line[7:11]
    month = line[12:14]
    day = line[15:17]
    
    return (int(year), int(month), int(day))

file_path = []

root_dir = './json/'
for root, dirs, files in os.walk(root_dir, onerror=None):
    for filename in files:
        if not any(s in filename for s in ['LOG', 'checkpoint', 'DS', 'edited']):
            file_path.append(os.path.join(root, filename))

for file in sorted(file_path, key=sorterkey):
    f = open(file)
    jData = json.loads(f.read())

    m = 9
    d = 14

    day, month, year = jData['timestamp'].split('-', 1)[0].split('.')
    if int(year) == 2024 and int(month) >= m:
        if int(month) == m and int(day) >= d:
            pass
        elif int(month) == m:
            continue
    else:
        continue

    for i, j in jData.items():
        print(i, j)

    if 'staff' in jData.keys():
        print(jData['staff'])
        if input('continue?') == 'yes':
            continue
        else:
            pass

    staff = []
    i = 0
    
    while True:
        i += 1
        if i > len(jData['hour']):
            break
            
        result = wich_one(input(f"{i}. {jData['hour'][i-1]}"))

        if result == 'noname':
            staff = 'false'
            break
            
        elif result not in STAFF:
            print('not in list.')
            i -= 1
        
        else:
            staff.append(str(result))

    jData['staff'] = staff

    with open(file, 'w+') as ff:
        json.dump(jData, ff)

In [180]:
import numpy as np
import os
from datetime import datetime as dt
import calendar
import warnings
import subprocess
import json
from urllib import request
import socket

from harvest_func import abort, git_update, fcalctip, is_connected


warnings.filterwarnings("ignore", category=RuntimeWarning)

l_ignore = ['LOG', 'checkpoint', 'DS', 'edited', 'TextIOWrapper']

def sorterkey(line):
    year = line[7:11]
    month = line[12:14]
    day = line[15:17]
    time = line[22:27].replace('-', '')
    
    return (int(year), int(month), int(day), int(time))


def calculate_statistic(files):
    d_return = {
        "date": [],
        "weekday": [],
        "time": [],
        "hour": [],
        "ratio": [],
        "total": [],
        "bar": [],
        "card": [],
        "holiday": [],
    }
    for file in files:
        try:
            f = open(file)
            jData = json.loads(f.read())
        except IndexError:
            print(file)

        s_date, s_time = jData['timestamp'].split('-', 1)
        d_return["date"].append(s_date)
        d_return["weekday"].append(dt.strptime(s_date, "%d.%m.%Y").weekday())

        if int(s_time[:2]) > 17:
            d_return["time"].append('PM')
        else:
            d_return["time"].append('AM')

        d_return["hour"].append(jData["hour"])
        
        d_return["ratio"].append(float(jData["ratio"]))

        d_return["total"].append(float(jData["sum"]))

        try:
            d_return["bar"].append(float(jData["bar"]))
        except ValueError:
            d_return["bar"].append(jData["ratio"])

        try:
            d_return["card"].append(float(jData["card"]))
        except ValueError:
            d_return["card"].append(jData["ratio"])

        d_return["holiday"].append(jData["holiday"])
        
        f.close()

    d_return["bar"] = list(filter(lambda item: item != 'None', d_return["bar"]))
    d_return["card"] = list(filter(lambda item: item != 'None', d_return["card"]))

    return d_return

In [186]:
path = './json/'

flag = True
files_whole = []
files_half = []
l_dict = ['whole', 'half']

i = 0
for dirpath, dirnames, filenames in os.walk(path):
    for f in filenames:
        if not any(s in f for s in l_ignore):
            if 'whole' in f.lower():
                files_whole.append(os.path.join(dirpath, f))
            else:
                files_half.append(os.path.join(dirpath, f))
                
            flag = False

files_whole = sorted(files_whole, key=sorterkey)
files_half = sorted(files_half, key=sorterkey)

d_whole = calculate_statistic(files_whole)
d_half = calculate_statistic(files_half)
d_all = [d_whole, d_half]


for theme in ['total', 'ratio', 'bar', 'card']:
    print(theme)
    for d in d_all:
        print(np.mean(d[theme]), '+/-', np.std(d[theme]))

total
184.56960227272728 +/- 52.60548976883578
109.07333333333334 +/- 31.32958666745056
ratio
4.416238636363636 +/- 0.7742871133899729
5.12909604519774 +/- 1.1191004644500788
bar
63.541931818181816 +/- 37.65075581331048
44.90965536723164 +/- 19.799055370090134
card
88.46289772727273 +/- 54.18800498465332
57.43281920903955 +/- 27.051623374349525
